In [1]:
import os
import numpy as np
# import pandas as pd # use pandas for more functionality
import modin.pandas as pd # use modin for speed
import scipy.cluster.hierarchy as sch
from scipy.cluster.hierarchy import fcluster
import matplotlib.pyplot as plt

In [2]:
# get working directory as paerent directory of current directory
cwd = os.getcwd()
pwd = os.path.dirname(cwd)

In [3]:
# Load the data
dtw_distance_matrix = np.load(pwd + "/02_Clean_data/02_dtw_distance_matrix.npy")

In [4]:
# Perform hierarchical clustering using the distance matrix
linkage_matrix = sch.linkage(dtw_distance_matrix, method='average')

# Form flat clusters from the hierarchical clustering defined by the given linkage matrix
# Example: Forming 3 clusters
clusters = fcluster(linkage_matrix, t=3, criterion='maxclust')
